In [2]:
import fiftyone as fo
from fiftyone import ViewField as F
from tqdm import tqdm
import numpy as np

In [3]:
dataset_name = "mcity_fisheye_3_months"
dataset = fo.load_dataset(dataset_name)

vru_labels = ["skater",
                "child",
                "bicycle",
                "bicyclist",
                "cyclist",
                "bike",
                "rider",
                "motorcycle",
                "motorcyclist",
                "pedestrian",
                "person",
                "walker",
                "jogger",
                "runner",
                "skateboarder",
                "scooter",
                "delivery driver"]

vehicle_labels = ["vehicle",
                "car",
                "bus",
                "truck",
                "taxi",
                "van",
                "pickup truck",
                "trailer",
                "emergency vehicle",]

In [4]:
#Get fields of zero shot model predictions
pred_fields = []
dataset_schema = dataset.get_field_schema()
for field in dataset_schema:
    if "pred_" in field:
        pred_fields.append(field)

conditions = [
    (F(f"{field}") != None) &          # Field exists
    (F(f"{field}.detections") != []) &  # Field has detections
    F(f"{field}.detections.label").contains(vru_labels)  # Detections include "cat" or "dog"
    for field in pred_fields
]

view = dataset.match(F.any(conditions))

In [5]:
for field in tqdm(pred_fields, desc="generating view"):
    view = view.filter_labels(f"{field}.detections",F("label").is_in(vru_labels), only_matches=False)
n_samples = len(view)

generating view: 100%|██████████| 10/10 [00:00<00:00, 148.76it/s]


In [6]:
samples_detections = [] # List of lists of list [model][sample][detections]
for field in tqdm(pred_fields, desc = "Getting detection values"):
    field_detections = view.values(f"{field}.detections")  # list of lists of detections per sample
    samples_detections.append(field_detections)

Getting detection values: 100%|██████████| 10/10 [01:26<00:00,  8.65s/it]


In [95]:
# Cleanup previous runs
v51_tag = "vru_overlap"

for i in tqdm(range(n_samples)):
    for j in range(len(pred_fields)):
        detections = samples_detections[j][i]
        for k in range(len(detections)):
            if len(samples_detections[j][i][k].tags) > 0:
                print(samples_detections[j][i][k].tags)
            try:
                samples_detections[j][i][k].tags.remove(v51_tag)
            except:
                pass       

100%|██████████| 49298/49298 [00:01<00:00, 24868.79it/s]


In [53]:
def calculate_iou(box1, box2):
    # box format: [minx, miny, width, height] normalized between [0,1]
    x1_min, y1_min, w1, h1 = box1
    x2_min, y2_min, w2, h2 = box2
    x1_max, y1_max = x1_min + w1, y1_min + h1
    x2_max, y2_max = x2_min + w2, y2_min + h2

    # Calculate the intersection box
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    if inter_x_min < inter_x_max and inter_y_min < inter_y_max:
        inter_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    else:
        inter_area = 0

    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0

In [ ]:
agreement_threshold = 3
iou_threshold = 0.5

max_detections_per_model = max(len(detections) for model_detections in samples_detections for detections in model_detections)

v51_tag = "vru_overlap"
n_bboxes_agreed = 0
n_samples_agreed = 0

debug_filepath = "/media/dbogdoll/Datasets/midadvrb_3_months/images/train/gs_Geddes_Huron1_2023-09-02 09-29-50-665323.jpg"

for sample_index in tqdm(range(n_samples)):  # For each sample

    agreement_found = False
    all_bboxes = []                 # List of all bounding box detections per sample
    bbox_model_indices = []         # Track which model each bounding box belongs to
    bbox_detection_indices = []     # Track the index of each detection in the model's list

    for model_index, model_detections in enumerate(samples_detections):
        for detection_index, det in enumerate(model_detections[sample_index]):
            all_bboxes.append(det.bounding_box)
            bbox_model_indices.append(model_index)
            bbox_detection_indices.append(detection_index)
    n_bboxes = len(all_bboxes)
    involved_models = [set()] * n_bboxes    # Track all models involved in successful overlaps

    if n_bboxes == 0:
        continue

    # Calculate IoU for each pair of bounding boxes
    iou_matrix = np.zeros((n_bboxes, n_bboxes))
    involved_models_matrix = np.full((n_bboxes, n_bboxes), -1)
    for a in range(n_bboxes):
        for b in range(a + 1, n_bboxes):
                iou = calculate_iou(all_bboxes[a], all_bboxes[b])
                # Only compare detections of different models
                if bbox_model_indices[a] != bbox_model_indices[b]:
                    if iou > iou_threshold:
                        iou_matrix[a, b] = 1
                        iou_matrix[b, a] = 1
                        involved_models_matrix[a, b] = bbox_model_indices[b]    # Store model index that was compared to
                        involved_models_matrix[b, a] = bbox_model_indices[a]
                        
                        involved_models_matrix[a, a] = bbox_model_indices[a]    # Trick to also store the model itself, as the diagonal is not used (b = a+1, symmetry)
                        involved_models_matrix[b, b] = bbox_model_indices[b]
                    
    # Get number of overlapping detections by summing up the rows
    model_overlaps = iou_matrix.sum(axis=1)

    # Get number of involved models by finding unique values in rows
    # "-1" ist not an involved model
    # As "-1" is present at all times, these cancel out regarding the count (if this is too slow)
    involved_models = [np.unique(row) for row in involved_models_matrix]
    for index in range(len(involved_models)):
        involved_models[index] = involved_models[index][involved_models[index] != -1]

    #Checking that all arrays have the same lengths
    assert len(all_bboxes) == len(bbox_model_indices) == len(bbox_detection_indices) == len(involved_models) == len(model_overlaps), "Array lengths should be identical"

    # Store detections that pass the agreement threshold
    for overlaps_index, overlap in enumerate(model_overlaps):
        if len(involved_models[overlaps_index]) >= agreement_threshold:
            agreement_found = True
            model_index = bbox_model_indices[overlaps_index]
            det_index = bbox_detection_indices[overlaps_index]
            if v51_tag not in samples_detections[model_index][sample_index][det_index].tags:
                samples_detections[model_index][sample_index][det_index].tags.append(v51_tag)
            n_bboxes_agreed += 1
    
    if agreement_found == True:
        n_samples_agreed += 1

print(f"Found {n_bboxes_agreed} detections with {agreement_threshold} or more overlapping detections in {n_samples_agreed} samples")

100%|██████████| 49298/49298 [00:15<00:00, 3286.14it/s]

Found 171349 detections with 3 or more overlapping detections in 20424 samples


In [98]:
for field, field_detections in tqdm(zip(pred_fields,samples_detections), desc = "Saving results to dataset", total=len(pred_fields)):
    view.set_values(field + ".detections", field_detections) 

Saving results to dataset:   0%|          | 0/10 [00:00<?, ?it/s]

Saving results to dataset: 100%|██████████| 10/10 [01:18<00:00,  7.87s/it]
